In [2]:
from transformers import T5Tokenizer, T5EncoderModel
import torch
import pickle
import pandas as pd
import re
from tqdm import tqdm

C:\Users\aenea\Desktop\Studium\PP2\protfrag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
BATCH_SIZE = 8
DATAFRAME_PATH = "../data/custom_fragments2/datasets/random_equal_distribution/random_equal_val.csv"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

cuda


In [8]:
#df = pd.read_csv(DATAFRAME_PATH)
df = fragment_df
df.head()

,source_accession,sequence,fragment_length,is_fragment
0,A0A068B6Q6,PDGRNAAAKAFDLITPTVRKGCCSNPACILNNPNQCG,37,True
1,A0A0D4WTV1,EGAEQDGSERTDGGRPIWNIAHMVNNKQAIDKYLDKGANSVESDVS...,289,True
2,A2QC57,MAGPAPSGRTPSHAQSSLPSLPAHLQSDTHLTAHLASRFHVGLPTA...,1614,True
3,A2TK72,KREAEANRTPEQQIYDPYKYVETVFVVDKAMVTKYNGDLDKIKTRM...,233,True
4,A5YVK8,AVIPLKNQGKCGSCWAFSTVTTVESINQIRTGNLISLSEQQLVDCS...,184,True


In [9]:
len(df)

9280

In [10]:
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False)
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")
model.to(DEVICE)
model.eval()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5EncoderModel(
  (shared): Embedding(128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=4096, bias=False)
              (k): Linear(in_features=1024, out_features=4096, bias=False)
              (v): Linear(in_features=1024, out_features=4096, bias=False)
              (o): Linear(in_features=4096, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=16384, bias=False)
              (wo): Linear(in_features=16384, out_features=1024, bias=False)
              (dropout): Dropo

In [11]:
# Filter sequences by length and create a proper copy to avoid SettingWithCopyWarning
length_df = df[df["fragment_length"] <= 1024].copy()
length_df.head()

,source_accession,sequence,fragment_length,is_fragment
0,A0A068B6Q6,PDGRNAAAKAFDLITPTVRKGCCSNPACILNNPNQCG,37,True
1,A0A0D4WTV1,EGAEQDGSERTDGGRPIWNIAHMVNNKQAIDKYLDKGANSVESDVS...,289,True
3,A2TK72,KREAEANRTPEQQIYDPYKYVETVFVVDKAMVTKYNGDLDKIKTRM...,233,True
4,A5YVK8,AVIPLKNQGKCGSCWAFSTVTTVESINQIRTGNLISLSEQQLVDCS...,184,True
5,A6XH06,MPLNSLHNLERKPSKAWSTSCTAPAARLQASFSLQQEEPRQIRRSG...,581,True


In [12]:
length_df["sequence_split"] = length_df["sequence"].apply(
    lambda seq: " ".join(list(re.sub(r"[UZOB]", "X", seq)))
)
length_df.head()

,source_accession,sequence,fragment_length,is_fragment,sequence_split
0,A0A068B6Q6,PDGRNAAAKAFDLITPTVRKGCCSNPACILNNPNQCG,37,True,P D G R N A A A K A F D L I T P T V R K G C C ...
1,A0A0D4WTV1,EGAEQDGSERTDGGRPIWNIAHMVNNKQAIDKYLDKGANSVESDVS...,289,True,E G A E Q D G S E R T D G G R P I W N I A H M ...
3,A2TK72,KREAEANRTPEQQIYDPYKYVETVFVVDKAMVTKYNGDLDKIKTRM...,233,True,K R E A E A N R T P E Q Q I Y D P Y K Y V E T ...
4,A5YVK8,AVIPLKNQGKCGSCWAFSTVTTVESINQIRTGNLISLSEQQLVDCS...,184,True,A V I P L K N Q G K C G S C W A F S T V T T V ...
5,A6XH06,MPLNSLHNLERKPSKAWSTSCTAPAARLQASFSLQQEEPRQIRRSG...,581,True,M P L N S L H N L E R K P S K A W S T S C T A ...


In [13]:
tokenized_df = pd.DataFrame(data=length_df[["source_accession", "is_fragment", "sequence_split"]])
tokenized_df.head()

,source_accession,is_fragment,sequence_split
0,A0A068B6Q6,True,P D G R N A A A K A F D L I T P T V R K G C C ...
1,A0A0D4WTV1,True,E G A E Q D G S E R T D G G R P I W N I A H M ...
3,A2TK72,True,K R E A E A N R T P E Q Q I Y D P Y K Y V E T ...
4,A5YVK8,True,A V I P L K N Q G K C G S C W A F S T V T T V ...
5,A6XH06,True,M P L N S L H N L E R K P S K A W S T S C T A ...


In [14]:
tokenized_input = []

In [15]:
for i in tqdm(range(0, len(tokenized_df), BATCH_SIZE)):
    batch = tokenized_df.loc[i:i+BATCH_SIZE - 1]

    encoded = tokenizer.batch_encode_plus(
        batch["sequence_split"],
        add_special_tokens=True,
        padding="longest",
        return_tensors="pt"
    )

    output = []

    for acc_id, is_fragment, input_ids, attention_mask in zip(batch["source_accession"], batch["is_fragment"], encoded["input_ids"], encoded["attention_mask"]):
        output.append((acc_id, is_fragment, input_ids, attention_mask))

    tokenized_input.append(output)

100%|██████████| 1144/1144 [00:01<00:00, 642.63it/s]


In [16]:
tokenized_input[0]

[('A0A068B6Q6',
  True,
  tensor([13, 10,  5,  8, 17,  3,  3,  3, 14,  3, 15, 10,  4, 12, 11, 13, 11,  6,
           8, 14,  5, 22, 22,  7, 17, 13,  3, 22, 12,  4, 17, 17, 13, 17, 16, 22,
           5,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [21]:
# Initialize as a list (not dict) since we'll use append
all_embeddings = []

In [22]:
# Create embeddings directory if it doesn't exist
import os
os.makedirs("./true_fragment_embeddings", exist_ok=True)
print("Embeddings directory ready")

Embeddings directory ready


In [23]:
# Test cell - checking embedding generation for a single sequence
with torch.no_grad():
    for (batch_num, entries) in tqdm(enumerate(tokenized_input), total=len(tokenized_input)):
        for acc_id, is_fragment, input_ids, attention_mask in entries:
            # Add batch dimension [seq_len] -> [1, seq_len]
            embedding = model(
                input_ids=input_ids.unsqueeze(0).to(DEVICE),
                attention_mask=attention_mask.unsqueeze(0).to(DEVICE)
            ).last_hidden_state
            print(f"Accession ID: {acc_id}")
            print(f"Fragment: {is_fragment}")
            print(f"Embedding shape: {embedding.shape}")  # Should be [1, seq_len, 1024]
            print(f"Attention mask shape: {attention_mask.shape}")  # Should be [seq_len]
            break
        break

  0%|          | 0/1144 [00:00<?, ?it/s]

Accession ID: A0A068B6Q6
Fragment: True
Embedding shape: torch.Size([1, 582, 1024])
Attention mask shape: torch.Size([582])


In [24]:
# Main embedding generation with mean pooling
with torch.no_grad():
    for (batch_num, entries) in tqdm(enumerate(tokenized_input), total=len(tokenized_input)):
        # Collect batch data
        acc_ids = []
        fragment_status = []
        batch_input_ids = []
        batch_attention_masks = []
        
        for (acc_id, is_fragment, input_ids, attention_mask) in entries:
            acc_ids.append(acc_id)
            fragment_status.append(is_fragment)
            batch_input_ids.append(input_ids)
            batch_attention_masks.append(attention_mask)
        
        # Stack into proper batches [batch_size, seq_len]
        batch_input_ids = torch.stack(batch_input_ids).to(DEVICE)
        batch_attention_masks = torch.stack(batch_attention_masks).to(DEVICE)
        
        # Get embeddings [batch_size, seq_len, embedding_dim]
        embeddings = model(
            input_ids=batch_input_ids,
            attention_mask=batch_attention_masks
        ).last_hidden_state
        
        # Mean pooling: average over sequence length, weighted by attention mask
        # Expand attention mask to match embedding dimensions
        mask = batch_attention_masks.unsqueeze(-1).expand(embeddings.size()).float()
        
        # Sum embeddings where attention mask is 1
        summed = torch.sum(embeddings * mask, dim=1)
        # Count how many positions were summed (sequence lengths)
        lengths = torch.sum(mask, dim=1)
        # Average: [batch_size, embedding_dim]
        prot_embeds = summed / lengths
        
        # Store embeddings with their accession IDs
        for i, (acc_id, is_fragment) in enumerate(zip(acc_ids, fragment_status)):
            all_embeddings.append((acc_id, is_fragment, prot_embeds[i].cpu()))
        
        # Progress logging
        if batch_num % 100 == 0:
            print(f"Processed {batch_num * BATCH_SIZE} sequences...")
        
        # Periodic saving
        if batch_num % 1000 == 0 and batch_num > 0:
            with open(f"./true_fragment_embeddings/embeddings_{batch_num}.pkl", "wb") as p:
                pickle.dump(all_embeddings, p)
            print(f"Saved checkpoint at batch {batch_num}")

# Save final embeddings
with open(f"./true_fragment_embeddings/final_embeddings.pkl", "wb") as p:
    pickle.dump(all_embeddings, p)
    
print(f"\nComplete! Generated {len(all_embeddings)} embeddings.")

  0%|          | 1/1144 [00:00<15:55,  1.20it/s]

Processed 0 sequences...


  9%|▉         | 101/1144 [01:34<14:12,  1.22it/s] 

Processed 800 sequences...


 18%|█▊        | 201/1144 [02:49<10:57,  1.44it/s]  

...

Saved checkpoint at batch 1000


 96%|█████████▋| 1103/1144 [10:09<00:02, 19.73it/s]

Processed 8800 sequences...


100%|██████████| 1144/1144 [10:11<00:00,  1.87it/s]


Complete! Generated 9022 embeddings.


In [20]:
# Verification: Load and inspect embeddings
with open("./embeddings/final_embeddings.pkl", "rb") as p:
    loaded_embeddings = pickle.load(p)

print(f"Total embeddings: {len(loaded_embeddings)}")
print(f"\nFirst embedding:")
print(f"  Accession ID: {loaded_embeddings[2][0]}")
print(f"  Embedding shape: {loaded_embeddings[2][2].shape}")
print(f"  Embedding type: {type(loaded_embeddings[2][2])}")
print(f"  Is Fragment: {loaded_embeddings[2][1]}")
print(f"\nExpected: torch.Size([1024]) for ProtT5-XL")

Total embeddings: 452946

First embedding:
  Accession ID: P02607
  Embedding shape: torch.Size([1024])
  Embedding type: <class 'torch.Tensor'>
  Is Fragment: False

Expected: torch.Size([1024]) for ProtT5-XL


In [40]:
df[df["source_accession"] == "A3Q8S8"]

,source_accession,fragment_type,sequence,is_fragment,fragment_length,source_length
0,A3Q8S8,mixed,MNGEKVKRLSTLAETLPIQVITPESFSLLFEGPKSRRQFIDWGAFH...,True,233,360
60607,A3Q8S8,complete,MSLSRLHIDSFRNIASAQLQLGDGLNLIYGQNGSGKTSILEAIFFL...,False,360,360


In [46]:
bool(tokenized_df.loc[0]["is_fragment"]) == True

True

In [43]:
tokenized_df.loc[60607]["is_fragment"]

np.False_

In [36]:
# create embedding df
csv_test = "\n".join(f"{acc_id},{",".join(str(v.item()) for v in embedding)}" for acc_id, embedding in loaded_embeddings)

with open("test.csv", "w") as f:
    f.write(csv_test)

In [37]:
len(csv_test)

9777332392

In [39]:
header = "acc_id," + ",".join(str(i) for i in range(1024))
header

'acc_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,27

In [48]:
len(tokenized_df)

465583

In [47]:
# add fragment label to list and store are pickle
annotated_embeddings = []

for i, (acc_id, embedding) in enumerate(loaded_embeddings):
    annotation = bool(tokenized_df.loc[i]["is_fragment"])
    annotated_embeddings.append((acc_id, embedding, annotation))

KeyError: 42

## generation of real fragments

In [4]:
fragment_df = pd.read_csv("./uniprotkb_reviewed_true_AND_fragment_tr_2025_12_01.tsv", sep="\t")
fragment_df

,entry,sequence
0,A0A068B6Q6,PDGRNAAAKAFDLITPTVRKGCCSNPACILNNPNQCG
1,A0A0D4WTV1,EGAEQDGSERTDGGRPIWNIAHMVNNKQAIDKYLDKGANSVESDVS...
2,A2QC57,MAGPAPSGRTPSHAQSSLPSLPAHLQSDTHLTAHLASRFHVGLPTA...
3,A2TK72,KREAEANRTPEQQIYDPYKYVETVFVVDKAMVTKYNGDLDKIKTRM...
4,A5YVK8,AVIPLKNQGKCGSCWAFSTVTTVESINQIRTGNLISLSEQQLVDCS...
...,...,...
9275,Q9R4P1,VQIFVRDNNVDQALKALK
9276,Q9R4P4,TKIADLRSQTVDQLSDXLXKL
9277,Q9R4P6,VQIFVXDNNVDQALK
9278,Q9R5V8,MKATELREKSAQQLNXQLL


In [5]:
fragment_df["fragment_length"] = [len(seq) for seq in fragment_df["sequence"]]
fragment_df

,entry,sequence,fragment_length
0,A0A068B6Q6,PDGRNAAAKAFDLITPTVRKGCCSNPACILNNPNQCG,37
1,A0A0D4WTV1,EGAEQDGSERTDGGRPIWNIAHMVNNKQAIDKYLDKGANSVESDVS...,289
2,A2QC57,MAGPAPSGRTPSHAQSSLPSLPAHLQSDTHLTAHLASRFHVGLPTA...,1614
3,A2TK72,KREAEANRTPEQQIYDPYKYVETVFVVDKAMVTKYNGDLDKIKTRM...,233
4,A5YVK8,AVIPLKNQGKCGSCWAFSTVTTVESINQIRTGNLISLSEQQLVDCS...,184
...,...,...,...
9275,Q9R4P1,VQIFVRDNNVDQALKALK,18
9276,Q9R4P4,TKIADLRSQTVDQLSDXLXKL,21
9277,Q9R4P6,VQIFVXDNNVDQALK,15
9278,Q9R5V8,MKATELREKSAQQLNXQLL,19


In [6]:
fragment_df = fragment_df.rename(columns={"entry": "source_accession"})
fragment_df

,source_accession,sequence,fragment_length
0,A0A068B6Q6,PDGRNAAAKAFDLITPTVRKGCCSNPACILNNPNQCG,37
1,A0A0D4WTV1,EGAEQDGSERTDGGRPIWNIAHMVNNKQAIDKYLDKGANSVESDVS...,289
2,A2QC57,MAGPAPSGRTPSHAQSSLPSLPAHLQSDTHLTAHLASRFHVGLPTA...,1614
3,A2TK72,KREAEANRTPEQQIYDPYKYVETVFVVDKAMVTKYNGDLDKIKTRM...,233
4,A5YVK8,AVIPLKNQGKCGSCWAFSTVTTVESINQIRTGNLISLSEQQLVDCS...,184
...,...,...,...
9275,Q9R4P1,VQIFVRDNNVDQALKALK,18
9276,Q9R4P4,TKIADLRSQTVDQLSDXLXKL,21
9277,Q9R4P6,VQIFVXDNNVDQALK,15
9278,Q9R5V8,MKATELREKSAQQLNXQLL,19


In [7]:
fragment_df["is_fragment"] = True
fragment_df

,source_accession,sequence,fragment_length,is_fragment
0,A0A068B6Q6,PDGRNAAAKAFDLITPTVRKGCCSNPACILNNPNQCG,37,True
1,A0A0D4WTV1,EGAEQDGSERTDGGRPIWNIAHMVNNKQAIDKYLDKGANSVESDVS...,289,True
2,A2QC57,MAGPAPSGRTPSHAQSSLPSLPAHLQSDTHLTAHLASRFHVGLPTA...,1614,True
3,A2TK72,KREAEANRTPEQQIYDPYKYVETVFVVDKAMVTKYNGDLDKIKTRM...,233,True
4,A5YVK8,AVIPLKNQGKCGSCWAFSTVTTVESINQIRTGNLISLSEQQLVDCS...,184,True
...,...,...,...,...
9275,Q9R4P1,VQIFVRDNNVDQALKALK,18,True
9276,Q9R4P4,TKIADLRSQTVDQLSDXLXKL,21,True
9277,Q9R4P6,VQIFVXDNNVDQALK,15,True
9278,Q9R5V8,MKATELREKSAQQLNXQLL,19,True
